In [1]:
import ccxt
from datetime import time
import asyncio

### API Keys Setup
#### Required Credentials
- The API credentials usually include the following:

    - **apiKey**. This is your public API Key and/or Token. This part is non-secret, it is included in your request header or body and sent over HTTPS in open text to identify your request. It is often a string in Hex or Base64 encoding or an UUID identifier.
    - **secret**. This is your private key. Keep it secret, don't tell it to anybody. It is used to sign your requests locally before sending them to exchanges. The secret key does not get sent over the internet in the request-response process and should not be published or emailed. It is used together with the nonce to generate a cryptographically strong signature. That signature is sent with your public key to authenticate your identity. Each request has a unique nonce and therefore a unique cryptographic signature.
    - **uid**. Some exchanges (not all of them) also generate a user id or uid for short. It can be a string or numeric literal. You should set it, if that is explicitly required by your exchange. See their docs for details.
    - **password**. Some exchanges (not all of them) also require your password/phrase for trading. You should set this string, if that is explicitly required by your exchange. See their docs for details.

#### Credential Validation
- For checking if the user has supplied all the required credentials the Exchange base class has a method called **exchange.checkRequiredCredentials()** or **exchange.check_required_credentials()**. Calling that method will throw an **AuthenticationError**, if some of the credentials are missing or empty. The **Exchange** base class also has property **exchange.requiredCredentials** that allows a user to see which credentials are required for this or that exchange, as shown below:

In [ ]:
exchange = ccxt.binance()
print(exchange.requiredCredentials())  # print required credentials

In [ ]:
exchange.check_required_credentials()   # raises AuthenticationError

#### Configuring API Keys
- To set up an exchange for trading just assign the API credentials to an existing exchange instance or pass them to exchange contructor upon instantiation, like so:

In [ ]:
# any time
bitfinex = ccxt.bitfinex()
bitfinex.apiKey = 'YOUR_BFX_API_KEY'
bitfinex.secret = 'YOUR_BFX_SECRET_KEY'

In [ ]:
# Upon instantiation
hitbtc = ccxt.hitbtc({
    'apiKey': 'YOUR_BFX_API_KEY',
    'secret': 'YOUR_BFX_SECRET_KEY'
})

In [ ]:
# from variable id
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET'
})

#### Overriding Nonce

In [ ]:
# 1: the shortest
binance = ccxt.binance({'nonce': ccxt.Exchange.milliseconds})

# 2: custom nonce
class MyKraken(ccxt.kraken):
    n = 1
    def nonce(self):
        return self.n += 1

# 3: milliseconds nonce
class MyBitfinex(ccxt.bitfinex):
    def nonce(self):
        return self.milliseconds()

# 4: milliseconds nonce inline
hitbtc = ccxt.hitbtc({'nonce': lambda: int(time.time()) * 1000})

# 5: milliseconds nonce
acx = ccxt.acx({'nonce': lambda: ccxt.Exchange.milliseconds()})

### Accounts
- You can get all the accounts associated with a profile by using the **fetchAccounts()** method

#### Accounts Structure
- The **fetchAccounts()** method will return a structure like shown below:

In [ ]:
accounts_structure = [
    {
        id: "s32kj302lasli3930",
        type: "main",
        name: "main",
        code: "USDT",
        info: { ... }
    },
    {
        id: "20f0sdlri34lf90",
        name: "customAccount",
        type: "margin",
        code: "USDT",
        info: { ... }
    },
    {
        id: "4oidfk40dadeg4328",
        type: "spot",
        name: "spotAccount32",
        code: "BTC",
        info: { ... }
    },
    ...
]

### Account Balance
- To query for balance and get the amount of funds available for trading or funds locked in orders, use the **fetchBalance** method:
#### Parameters
- **params** (Dictionary) Extra parameters specific to the exchange API endpoint (e.g. {"currency": "usdt"})
#### Returns
A **balance structure**

In [ ]:
balance_structure = {
    'info':  { ... },    # the original untouched non-parsed reply with details
    'timestamp': 1499280391811, # Unix Timestamp in milliseconds (seconds * 1000)
    'datetime': '2017-07-05T18:47:14.692Z', # ISO8601 datetime string with milliseconds

    #-------------------------------------------------------------------------
    # indexed by availability of funds first, then by currency

    'free':  {           # money, available for trading, by currency
        'BTC': 321.00,   # floats...
        'USD': 123.00,
        
    },

    'used':  { ... },    # money on hold, locked, frozen, or pending, by currency

    'total': { ... },    # total (free + used), by currency

    #-------------------------------------------------------------------------
    # indexed by currency first, then by availability of funds

    'BTC':   {           # string, three-letter currency code, uppercase
        'free': 321.00,   # float, money available for trading
        'used': 234.00,  # float, money on hold, locked, frozen or pending
        'total': 555.00, # float, total balance (free + used)
    },

    'USD':   {           # ...
        'free': 123.00,   # ...
        'used': 456.00,
        'total': 579.00,
    },

    
}

In [ ]:
print(exchange.fetch_balance())

### Querrying Orders
- The list of methods for querying orders consists of the following:

    - fetchCanceledOrders (symbol = undefined, since = undefined, limit = undefined, params = {})
    - fetchClosedOrder (id, symbol = undefined, params = {})
    - fetchClosedOrders (symbol = undefined, since = undefined, limit = undefined, params = {})
    - fetchOpenOrder (id, symbol = undefined, params = {})
    - fetchOpenOrders (symbol = undefined, since = undefined, limit = undefined, params = {})
    - fetchOrder (id, symbol = undefined, params = {})
    - fetchOrders (symbol = undefined, since = undefined, limit = undefined, params = {})

In [ ]:
# look into the .has property of the exchange to check above available querying methods
id = 'binance'
ex_class = getattr(ccxt, id)()
print(ex_class.has)

**NOTE**: not all exchanges provide means for filtering the lists of trades and orders by starting time, so, the support for since  and limit is exchange-specific. However, most exchanges do provide at least some alternative for "pagination" and "scrolling" which can be overrided with extra params argument.

#### By Order Id
- To get the details of a particular order by its id, use the fetchOrder() / fetch_order() method. Some exchanges also require a symbol even when fetching a particular order by id.

- The signature of the fetchOrder/fetch_order method is as follows:

In [ ]:
if exchange.has['fetchOrder']:
    cursor = 0
    while True:
        id = 'binance'
        symbol = 'BTC/USDT'
        params = {
            'cursor': cursor
        }
        order = await exchange.fetch_order(id, symbol=symbol, params=params)
        print(order)

#### Single Order
- Below are examples of using the fetchOrder method to get order info from an authenticated exchange instance:

In [ ]:
if exchange.has['fetchOrder']:
    order = exchange.fetch_order(id)
    print(order)

In [ ]:
# synchronous
import asyncio
import ccxt.async_support as ccxt

if exchange.has['fetchOrder']:
    order = asyncio.run(exchange.fetch_order(id))
    print(order)

#### All Orders

In [ ]:
if exchange.has['fetchOrders']:
    cursor = 0
    all_orders = []
    while True:
        since = exchange.milliseconds() * 60 * 60 * 1000   # last 1 hour
        limit = 10
        params = {
            'cursor': cursor
        }
        orders = exchange.fetchOrders(symbol=symbol, since=since, limit=limit, params=params)
        if len(orders) == 0:
            break
        else:
            cursor = exchange.last_response_headers['CB-AFTER']
            all_orders += orders

#### Open Orders

In [ ]:
if exchange.has['fetchOpenOrders']:
    cursor = 0
    all_open_orders = []
    while True:
        symbol = 'DOGE/USDT'
        since = exchange.milliseconds() * 60 * 60 * 1000
        limit = 10
        params = {
            'cursor': cursor
        }
        open_orders = exchange.fetch_open_orders(symbol=symbol, since=since, limit=limit, params=params)
        if len(open_orders):
            cursor = exchange.last_response_headers['CB-AFTER']
            all_open_orders += open_orders
        else:
            break

#### Closed Orders

In [ ]:
if exchange.has['fetchClosedOrders']:
    closed_orders = exchange.fetch_closed_orders(symbol=symbol, since=since, limit=limit, params=params)
    print(closed_orders)

#### Order Structure
- Most of methods returning orders within ccxt unified API will yield an order structure as described below:

In [ ]:
order_structure = {
    'id':                '12345-67890:09876/54321', // string
    'clientOrderId':     'abcdef-ghijklmnop-qrstuvwxyz', // a user-defined clientOrderId, if any
    'datetime':          '2017-08-17 12:42:48.000', // ISO8601 datetime of 'timestamp' with milliseconds
    'timestamp':          1502962946216, // order placing/opening Unix timestamp in milliseconds
    'lastTradeTimestamp': 1502962956216, // Unix timestamp of the most recent trade on this order
    'status':      'open',        // 'open', 'closed', 'canceled', 'expired', 'rejected'
    'symbol':      'ETH/BTC',     // symbol
    'type':        'limit',       // 'market', 'limit'
    'timeInForce': 'GTC',         // 'GTC', 'IOC', 'FOK', 'PO'
    'side':        'buy',         // 'buy', 'sell'
    'price':        0.06917684,   // float price in quote currency (may be empty for market orders)
    'average':      0.06917684,   // float average filling price
    'amount':       1.5,          // ordered amount of base currency
    'filled':       1.1,          // filled amount of base currency
    'remaining':    0.4,          // remaining amount to fill
    'cost':         0.076094524,  // 'filled' * 'price' (filling price used where available)
    'trades':     [ ... ],        // a list of order trades/executions
    'fee': {                      // fee info, if available
        'currency': 'BTC',        // which currency the fee is (usually quote)
        'cost': 0.0009,           // the fee amount in that currency
        'rate': 0.002,            // the fee rate (if available)
    },
    'info': { ... },              // the original unparsed order structure as is
}

##### timeInForce
- The timeInForce field may be undefined/None/null if not specified by the exchange. - The unification of timeInForce is a work in progress.

- Possible values for thetimeInForce field:

    - **'GTC'** = *Good Till Cancel(ed)*, the order stays on the orderbook until it is matched or canceled.
    - **'IOC'** = *Immediate Or Cancel*, the order has to be matched immediately and filled either partially or completely, the unfilled remainder is canceled (or the entire order is canceled).
    - **'FOK'** = *Fill Or Kill*, the order has to get fully filled and closed immediately, otherwise the entire order is canceled.
    - **'PO'** = *Post Only*, the order is either placed as a maker order, or it is canceled. This means the order must be placed on orderbook for at at least time in an unfilled state. The unification of **PO** as a **timeInForce** option is a work in progress with unified exchanges having **exchange.has['createPostOnlyOrder'] == True**.

### Placing Orders

- An example of finding the **contractSize**:

In [ ]:
await exchange.loadMarkets()
symbol = 'BTC/USDT:USDT'
market = exchange.market(symbol)
print(market['contractSize'])

# Let's say you want to convert 0.5 BTC to the number of contracts:
number_contracts = round((0.5 * 1) / market['contractSize'])

#### Limit Orders
- Limit orders placed on the order book of the exchange for a price specified by the trader. They are fullfilled(closed) when there are no orders in the same market at a better price, and another trader creates a market order or an opposite order for a price that matches or exceeds the price of the limit order.

- Limit orders may not be fully filled. This happens when the filling order is for a smaller amount than the amount specified by the limit order.

In [ ]:
sym = 'BTC/USDT'
price = 55001.0
amount = 0.01813614
params = {
    'settle': 'usdt'
}

exchange.create_limit_buy_order(symbol=symbol, amount=amount, price=price, params=params)
exchange.create_limit_sell_order(symbol=symbol, amount=amount, price=price, params=params)

In [ ]:
# using general createLimitOrder and side = 'buy' or 'sell'
exchange.create_limit_order(
    symbol=symbol, 
    side='buy', 
    amount=amount, 
    price=price, 
    params=params
)

In [ ]:
# using general createMarketOrder and side = 'buy' or 'sell'
exchange.create_market_order(
    symbol=symbol, 
    side='buy', 
    amount=amount, 
    params=params
)

In [ ]:
# using general createOrder, type = 'limit' and side = 'buy' or 'sell'
exchange.create_order(
    symbol=symbol, 
    type='limit', 
    side='sell', 
    amount=amount, 
    price=price, 
    params=params
)

In [ ]:
if exchange.has['createMarketOrder']:
    while True:
        symbol = 'DOGE/BTC'
        amount = 0.01813614
        # price = 55001.0
        params = {
            'settle': 'usdt'
        }
        order = exchange.create_market_order(
            symbol = symbol,
            type = 'limit',
            side = 'sell',
            amount = amount,
            # price = price, # this will be ignored in creating market order   
            params = params
        )

        print(order)

- Limit price orders are also known as *limit orders*. Some exchanges accept limit orders only. Limit orders require a price (rate per unit) to be submitted with the order. The exchange will close limit orders if and only if market price reaches the desired level.

In [ ]:
# camelCaseStyle
exchange.createLimitBuyOrder (symbol, amount, price, params)
exchange.createLimitSellOrder (symbol, amount, price, params)

# underscore_style
exchange.create_limit_buy_order (symbol, amount, price, params)
exchange.create_limit_sell_order (symbol, amount, price, params)

#### Market Buys

In [ ]:
exchange = ccxt.cex({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'SECRET',
    'options': {
        'createMarketBuyOrderRequiresPrice': False,  # switch off
    }
})

exchange.options['createMarketBuyOrderRequiresPrice'] = False

async def function():
    pass

#### Stop Orders
- Coming from traditional trading, the term "Stop" order has been a bit ambigious, so instead of it, in CCXT we use term "Trigger" order. When symbol's price reaches your "trigger"("stop") price, the order is activated - if you had chosen *market order*, then it will be executed immediately, if you have chosen limit order, it will be placed in the orderbook.

- We have different classification of trigger orders:

    - stand-alone *Trigger order* to buy/sell coin (open/close position)
    - stand-alone *Stop-Loss* or *Take-Profit* order which are only designed to close an open position.
    - an attached *Stop-Loss* or *Take-Profit* order into a primary order (Conditional Trigger Order).
##### Trigger Orders
- Traditional "stop" order (which you might see across exchanges' websites) is now called "trigger" order across CCXT library. Implemented by adding a **triggerPrice** parameter. They are independent basic trigger orders that can open or close a position.

    - Typically, it is activated when price of the underlying asset/contract crosses the **triggerPrice** from any direction. However, some exchanges' API require to set **triggerDirection** too, which triggers order depending whether price is above or below **triggerPrice**. For example, if you want to trigger limit order (buy 0.1 **ETH** at limit price **1500**) once pair price crosses **1700**:

In [ ]:
sym = 'ETH/USDT'
price = 1500
amount = 0.1
params = {
    'triggerPrice': '1700'
}

order = exchange.create_order(symbol, 'market', 'buy', amount, price, params)

In [ ]:
params = {
    'triggerPrice': 1700,
    'triggerDirection': 'above',   # order will be triggered when price is above 1700
}

##### Stop Loss Orders
- The same as Trigger Orders, but the direction matters. Implemented by specifying a **stopLossPrice** parameter (for the stop loss *triggerPrice*), and also automatically implemented **triggerDirection** on behalf of user, so instead of regular Trigger Order, you can use this as an alternative.

- Stop Loss orders are activated when the price of the underlying asset/contract:

    - drops below the stopLossPrice from above, for sell orders. (eg: to close a long position, and avoid further losses)
    - rises above the stopLossPrice from below, for buy orders (eg: to close a short position, and avoid further losses)

In [ ]:
params = {
    'stopLossPrice': 55.45,   # your stop loss price
}
order = exchange.create_order(symbol, 'market', 'sell', amount, price, params)

##### Take Profit Orders
- The same as Stop Loss Orders, but the direction matters. Implemented by specifying a **takeProfitPrice** parameter (for the take profit triggerPrice).

- Take Profit orders are activated when the price of the underlying:

    - rises above the takeProfitPrice from below, for sell orders (eg: to close a long position, at a profit)
    - drops below the takeProfitPrice from above, for buy orders (eg: to close a short position, at a profit)

In [ ]:
# for a take profit order
params = {
    'takeProfitPrice': 120.45    # your take profit price
}
order = exchange.create_order(symbol, 'market', 'buy', amount, price, params)

##### StopLoss And TakeProfit Orders Attached To A Position
- **Take Profit / Stop** Loss Orders which are tied to a position-opening primary order. - Implemented by supplying a dictionary parameters for **stopLoss** and **takeProfit** describing each respectively.

    - By default stopLoss and takeProfit orders will be the same magnitude as primary order but in the opposite direction.
    - Attached trigger orders are conditional on the primary order being executed.
    Not supported by all exchanges.
    - Both **stopLoss** and **takeProfit** or either can be supplied, this depends on exchange.

In [ ]:
symbol = 'ETH/BTC'
type = 'limit'  # or 'market'
side = 'buy'
amount = 123.45  # your amount
price = 115.321  # your price
params = {
    'stopLoss': {
        'type': 'limit',  # or 'market', this field is not necessary if limit price is specified
        'price': 100.33,  # limit price for a limit stop loss order
        'triggerPrice': 101.25,
    },
    'takeProfit': {
        'type': 'market',  # or 'limit', this field is not necessary if limit price is specified
        # no limit price for a market take profit order
        # 'price': 160.33,  # this field is not necessary for a market take profit order
        'triggerPrice': 150.75,
    }
}
order = exchange.create_order (symbol, type, side, amount, price, params)

##### Trailing Orders
- Trailing Orders trail behind an open position. Implemented by supplying float parameters for **trailingPercent** or **trailingAmount**.

- A trailing order continually adjusts the order price at a fixed percent or fixed quote amount away from the current market price.
- A trailing order trails behind a position as it moves in one direction, but not in the opposite direction.
- If the position value rises, the trailing order changes, but if the position value drops the trailing order stays the same until the order is executed.
- A trailing order can be placed independently after opening a position.
- Implemented by filling in either the **trailingPercent** or **trailingAmount** parameter depending on the exchange.
- The price argument can be used as the **trailingTriggerPrice**, and the type argument can be used to differentiate between limit and market trailing orders if needed.

In [ ]:
symbol = 'BTC/USDT:USDT'
type = 'market'
side = 'sell'
amount = 1.0
price = None
params = {
    'trailingPercent': 1.0, # percentage away from the current market price 1.0 is equal to 1%
    # 'trailingAmount': 100.0, # quote amount away from the current market price
    # 'trailingTriggerPrice': 44500.0, # the price to trigger activating a trailing stop order
    # 'reduceOnly': True, # set to True if you want to close a position, set to False if you want to open a new position
}
order = exchange.create_order (symbol, type, side, amount, price, params)

##### Custom Order Params
- Some exchanges allow you to specify optional parameters for your order. You can pass your optional parameters and override your query with an associative array using the **params** argument to your unified API call. All custom params are exchange-specific, of course, and aren't interchangeable, do not expect those custom params for one exchange to work with another exchange.

In [ ]:
kraken = ccxt.kraken()
kraken.create_market_buy_order(symbol, {'trading_agreement': 'agree'})

- The user can specify a custom **clientOrderId** field can be set upon placing orders with the **params**. Using the **clientOrderId** one can later distinguish between own orders. This is only available for the exchanges that do support **clientOrderId** at this time. For the exchanges that don't support it will either throw an error upon supplying the **clientOrderId** or will ignore it setting the **clientOrderId** to **undefined/None/null**.

In [ ]:
kraken.create_order(symbol, type, side, amount, price, {
    'clientOrderId': 'World',
})

##### Editing Orders
- To edit an order, you can use the editOrder method

In [ ]:
kraken.edit_order(id, symbol, type, side, amount, price = 130, params = {})

### Personal Trades

In [ ]:
# fetch_my_trades(symbol=None, since=None, limit=None, params={})

if exchange.has['fetchMyTrades']:
    exchange.fetch_my_trades(symbol=None, since=None, limit=None, params={})

In [ ]:
trade_structure = {
    'info':         { ... },                    // the original decoded JSON as is
    'id':           '12345-67890:09876/54321',  // string trade id
    'timestamp':    1502962946216,              // Unix timestamp in milliseconds
    'datetime':     '2017-08-17 12:42:48.000',  // ISO8601 datetime with milliseconds
    'symbol':       'ETH/BTC',                  // symbol
    'order':        '12345-67890:09876/54321',  // string order id or undefined/None/null
    'type':         'limit',                    // order type, 'market', 'limit' or undefined/None/null
    'side':         'buy',                      // direction of the trade, 'buy' or 'sell'
    'takerOrMaker': 'taker',                    // string, 'taker' or 'maker'
    'price':        0.06917684,                 // float price in quote currency
    'amount':       1.5,                        // amount of base currency
    'cost':         0.10376526,                 // total cost, `price * amount`,
    'fee':          {                           // provided by exchange or calculated by ccxt
        'cost':  0.0015,                        // float
        'currency': 'ETH',                      // usually base currency for buys, quote currency for sells
        'rate': 0.002,                          // the fee rate (if available)
    },
    'fees': [                                   // an array of fees if paid in multiple currencies
        {                                       // if provided by exchange or calculated by ccxt
            'cost':  0.0015,                    // float
            'currency': 'ETH',                  // usually base currency for buys, quote currency for sells
            'rate': 0.002,                      // the fee rate (if available)
        },
    ],
}

### Trades By Order Id

In [ ]:
# fetch_order_trades(id, symbol=None, since=None, limit=None, params={})
order_id = 'binance'
since = exchange.millisecobds() * 60 * 60 * 100
limit = 10
if exchange.has['fetchOrderTrades']:
    exchange.fetch_order_trades(
        order_id, 
        symbol='ETH/USDT', 
        since=since, 
        limit=limit, 
        params={}
    )

### Ledger
- The ledger is simply the history of changes, actions done by the user or operations that altered the user's balance in any way, that is, the history of movements of all funds from/to all accounts of the user which includes

    - deposits and withdrawals (funding)
    - amounts incoming and outcoming in result of a trade or an order
    - trading fees
    - transfers between accounts
    - rebates, cashbacks and other types of events that are subject to accounting.
- Data on ledger entries can be retrieved using

    - **fetchLedgerEntry()** for a ledger entry
    - **fetchLedger( code )** for multiple ledger entries of the same currency
    - **fetchLedger()** for all ledger entries

### Ledger Entry Structure

In [ ]:
ledger_entry_structure = {
    'id': 'hqfl-f125f9l2c9',                // string id of the ledger entry, e.g. an order id
    'direction': 'out',                     // or 'in'
    'account': '06d4ab58-dfcd-468a',        // string id of the account if any
    'referenceId': 'bf7a-d4441fb3fd31',     // string id of the trade, transaction, etc...
    'referenceAccount': '3146-4286-bb71',   // string id of the opposite account (if any)
    'type': 'trade',                        // string, reference type, see below
    'currency': 'BTC',                      // string, unified currency code, 'ETH', 'USDT'...
    'amount': 123.45,                       // absolute number, float (does not include the fee)
    'timestamp': 1544582941735,             // milliseconds since epoch time in UTC
    'datetime': "2018-12-12T02:49:01.735Z", // string of timestamp, ISO8601
    'before': 0,                            // amount of currency on balance before
    'after': 0,                             // amount of currency on balance after
    'status': 'ok',                         // 'ok, 'pending', 'canceled'
    'fee': {                                // object or undefined
        'cost': 54.321,                     // absolute number on top of the amount
        'currency': 'ETH',                  // string, unified currency code, 'ETH', 'USDT'...
    },
    'info': { ... },                        // raw ledger entry as is from the exchange
}

### Deposit
- In order to deposit funds to an exchange you must get an address from the exchange for the currency you want to deposit there. Most of exchanges will create and manage those addresses for the user.

- Data on deposits made to an account can be retrieved using

    - **fetchDeposit()** for a single deposit
    - **fetchDeposits( code )** for multiple deposits of the same currency
    - **fetchDeposits()** for all deposits to an account

In [ ]:
exchange.fetch_deposits(code = 'USDT', since = None, limit = None, params = {})

### Withdrawal
- The withdraw method can be used to withdraw funds from an account

In [ ]:
exchange.withdraw(code='USDT', amount=150, address=None, tag=None, params={})

#### Returns
- A **transaction structure**

- Data on withdrawals made to an account can be retrieved using

    * **fetchWithdrawal()** for a single withdrawal
    * **fetchWithdrawals( code )** for multiple withdrawals of the same currency
    * **fetchWithdrawals()** for all withdrawals from an account

#### Parameters

- **id** (String) required Withdrawal id
- **code** (String) Unified CCXT currency code (e.g. "USDT")
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. *{"network": "TRX"})*

In [ ]:
exchange.fetch_withdrawal(id, code = None, params = {})

#### Parameters

- **code** (String) Unified CCXT currency code (e.g. *"USDT")*
- **since** (Integer) Timestamp (ms) of the earliest time to retrieve withdrawals for (e.g. *1646940314000*)
- **limit** (Integer) The number of **transaction structures** to retrieve (e.g. *5*)
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. *{"endTime": 1645807945000}*)

##### Returns

- An array of **transaction structures**

In [ ]:
exchange.fetch_withdrawals(code = None, since = None, limit = None, params = {})

### Deposit And Withdrawal Networks
- It is also possible to pass the parameters as the fourth argument with or without a specified tag

In [ ]:
exchange.withdraw(code=None, amount=amount, address=None, { 'tag': tag, 'network': 'ETH' })

#### Transaction Structure
- *deposit structure*
- *withdrawal structure*

In [ ]:
transaction_structure = {
    'info':      { ... },    // the JSON response from the exchange as is
    'id':       '123456',    // exchange-specific transaction id, string
    'txid':     '0x68bfb29821c50ca35ef3762f887fd3211e4405aba1a94e448a4f218b850358f0',
    'timestamp': 1534081184515,             // timestamp in milliseconds
    'datetime': '2018-08-12T13:39:44.515Z', // ISO8601 string of the timestamp
    'addressFrom': '0x38b1F8644ED1Dbd5DcAedb3610301Bf5fa640D6f', // sender
    'address':  '0x02b0a9b7b4cDe774af0f8e47cb4f1c2ccdEa0806', // "from" or "to"
    'addressTo': '0x304C68D441EF7EB0E2c056E836E8293BD28F8129', // receiver
    'tagFrom', '0xabcdef', // "tag" or "memo" or "payment_id" associated with the sender
    'tag':      '0xabcdef' // "tag" or "memo" or "payment_id" associated with the address
    'tagTo': '0xhijgklmn', // "tag" or "memo" or "payment_id" associated with the receiver
    'type':     'deposit',   // 'withdrawal' or 'transfer', string
    'amount':    1.2345,     // float (does not include the fee)
    'currency': 'ETH',       // a common unified currency code, string
    'status':   'pending',   // 'ok', 'failed', 'canceled', string
    'updated':   undefined,  // UTC timestamp of most recent status change in ms
    'comment':  'a comment or message defined by the user if any',
    'fee': {                 // the entire fee structure may be undefined
        'currency': 'ETH',   // a unified fee currency code
        'cost': 0.1234,      // float
        'rate': undefined,   // approximately, fee['cost'] / amount, float
    },
}

### fetchDeposits Examples

In [ ]:
# fetch_deposits(code = None, since = None, limit = None, params = {})

if exchange.has['fetchDeposits']:
    deposits = exchange.fetch_deposits(code='USDT', since=since, limit=limit, params={})
else:
    raise Exception(exchange.id + ' does not have the fetch_deposits method')

### fetchWithdrawals Examples

In [ ]:
# fetch_withdrawals(code = None, since = None, limit = None, params = {})

if exchange.has['fetchWithdrawals']:
    withdrawals = exchange.fetch_withdrawals(code='USDT', since=since, limit=limit, params={})
else:
    raise Exception(exchange.id + ' does not have the fetch_withdrawals method')

### fetchTransactions Examples

In [ ]:
# fetch_transactions(code = None, since = None, limit = None, params = {})

if exchange.has['fetchTransactions']:
    transactions = exchange.fetch_transactions(code='USDT', since=since, limit=limit, params={})
else:
    raise Exception(exchange.id + ' does not have the fetch_transactions method')

### Deposit Addresses
- The address for depositing can be either an already existing address that was created previously with the exchange or it can be created upon request. In order to see which of the two methods are supported, check the **exchange.has['fetchDepositAddress']** and **exchange.has['createDepositAddress']** properties.

#### Parameters

- **code** (String) required Unified CCXT currency code (e.g. "USDT")
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. {"endTime": 1645807945000})

In [ ]:
exchange.fetch_deposit_address (code="USDT", params = {})
exchange.create_deposit_address (code="USDT", params = {})

#### Parameters

- **code** ([String]) Array of unified CCXT currency codes. May or may not be required depending on the exchange (e.g. ["USDT", "BTC"])
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. {"endTime": 1645807945000})

In [ ]:
exchange.fetch_deposit_addresses (codes = None, params = {})

#### Parameters

- **code** (String) required Unified CCXT currency code (e.g. "USDT")
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. {"endTime": 1645807945000})

In [ ]:
exchange.fetch_deposit_addresses_by_network(code=None, params = {})

In [ ]:
#### Address Structure
address_structure = {
    'currency': currency, // currency code
    'network': network,   // a list of deposit/withdraw networks, ERC20, TRC20, BSC20 (see below)
    'address': address,   // address in terms of requested currency
    'tag': tag,           // tag / memo / paymentId for particular currencies (XRP, XMR, ...)
    'info': response,     // raw unparsed data as returned from the exchange
}

### Transfers
- The **transfer** method makes internal transfers of funds between accounts on the same exchange. This can include subaccounts or accounts of different types (**spot, margin, future**, ...). If an exchange is separated on CCXT into a spot and futures class (e.g. **binanceusdm, kucoinfutures**, ...), then the method **transferIn** may be available to transfer funds into the futures account, and the method **transferOut** may be available to transfer funds out of the futures account

#### Parameters

- **code** (String) Unified CCXT currency code (e.g. *"USDT"*)
- **amount** (Float) The amount of currency to transfer (e.g. *10.5*)
- **fromAccount** (String) The account to transfer funds from.
- **toAccount** (String) The account to transfer funds to.
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. *{"endTime": 1645807945000}*)
- **params.symbol** (String) Market symbol when transfering to or from a margin account (e.g. *'BTC/USDT'*)

In [ ]:
exchange.transfer(code=None, amount=amount, fromAccount, toAccount, params = {})

#### Transfer Structure

In [ ]:
transfer_structure = {
    info: { ... },
    id: "93920432048",
    timestamp: 1646764072000,
    datetime: "2022-03-08T18:27:52.000Z",
    currency: "USDT",
    amount: 11.31,
    fromAccount: "spot",
    toAccount: "future",
    status: "ok"
}

### Fees
- This section of the Unified CCXT API is under development.

- Fees are often grouped into two categories:

    - Trading fees. Trading fee is the amount payable to the exchange, usually a percentage of volume traded (filled).
    - Transaction fees. The amount payable to the exchange upon depositing and withdrawing as well as the underlying crypto transaction fees (tx fees).

- Because the fee structure can depend on the actual volume of currencies traded by the user, the fees can be account-specific. Methods to work with account-specific fees:

    - fetchTradingFee (symbol, params = {})
    - fetchTradingFees (params = {})
    - fetchDepositWithdrawFees (codes = undefined, params = {})
    - fetchDepositWithdrawFee(code, params = {})
- You call **fetchTradingFee/fetchTradingFees** to fetch the trading fees, **fetchDepositWithdrawFee/fetchDepositWithdrawFees** to fetch the deposit & withdraw fees.

#### Fee Structure
- Orders, private trades, transactions and ledger entries may define the following info in their **fee** field:

In [ ]:
fee_structure = {
    'currency': 'BTC', // the unified fee currency code
    'rate': percentage, // the fee rate, 0.05% = 0.0005, 1% = 0.01, ...
    'cost': feePaid, // the fee cost (amount * fee rate)
}

### Trading Fees

In [ ]:
exchange.calculate_fee(symbol, type, side, amount, price, takerOrMaker = 'taker', params = {})

- The **calculateFee** method will return a unified fee structure with precalculated fees for an order with specified params.

- Accessing trading fee rates should be done via the **.markets** property, like so:

In [ ]:
exchange.markets['ETH/BTC']['taker'] # taker fee rate for ETH/BTC
exchange.markets['BTC/USD']['maker'] # maker fee rate for BTC/USD

### Trading Fee Schedule
- Some exchanges have an endpoint for fetching the trading fee schedule, this is mapped to the unified methods **fetchTradingFees**, and ****fetchTradingFee**
    - **fetchTradingFee (symbol, params = {})**

#### Parameters
- **symbol** (String) required Unified market symbol (e.g. "BTC/USDT")
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. *{"currency": "quote"}*)

#### Trading Fee Structure

In [ ]:
trading_fee_structure = {
    'ETH/BTC': {
        'maker': 0.001,
        'taker': 0.002,
        'info': { ... },
        'symbol': 'ETH/BTC',
    },
    'LTC/BTC': {
        'maker': 0.001,
        'taker': 0.002,
        'info': { ... },
        'symbol': 'LTC/BTC',
    },
}

### Transaction Fees

In [ ]:
exchange.currencies['ETH']['fee'] # tx/withdrawal fee rate for ETH
exchange.currencies['BTC']['fee'] # tx/withdrawal fee rate for BTC

### Transaction Fee Schedule
- Some exchanges have an endpoint for fetching the transaction fee schedule, this is mapped to the unified methods

    - **fetchTransactionFee()** for a single transaction fee schedule
    - **fetchTransactionFees()** for all transaction fee schedules

#### Parameters
- **code** (String) required Unified CCXT currency code, required (e.g. "USDT")
- **params** (Dictionary) Parameters specific to the exchange API endpoint (e.g. {"type": "deposit"})
- **params.network** (String) Specify unified CCXT network (e.g. {"network": "TRC20"})

In [ ]:
exchange.fetch_transaction_fees(
    codes = None, 
    params = {'type': 'deposit'},
    params.network = {'network': 'TRC20'},
)

#### Transaction Fee Structure

In [ ]:
transaction_fee_structure = {
    'withdraw': {
        'BTC': 0.00001,
        'ETH': 0.001,
        'LTC': 0.0003,
    },
    'deposit': {
        'BTC': 0,
    },
    'info': { ... },
}

### Borrow Interest

In [ ]:
exchange.fetch_borrow_interest(code = None, symbol = None, since = None, limit = None, params = {'endTime': 1645807945000})

#### Borrow Interest Structure

In [ ]:
borrow_interest_structure = {
    account: 'BTC/USDT',                    # The market that the interest was accrued in
    currency: 'USDT',                       # The currency of the interest
    interest: 0.00004842,                   # The amount of interest that was charged
    interestRate: 0.0002,                   # The borrow interest rate
    amountBorrowed: 5.81,                   # The amount of currency that was borrowed
    timestamp: 1648699200000,               # The timestamp that the interest was charged
    datetime: '2022-03-31T04:00:00.000Z',   # The datetime that the interest was charged
    info: { ... }                           # Unparsed exchange response
}

### Borrow And Repay Margin
*margin only*

- To borrow and repay currency as a margin loan use **borrowMargin** and **repayMargin**.

In [ ]:
exchange.borrow_margin(code=None, amount=amount, symbol = None, params = {"rate": 0.002})
exchange.repay_margin (code=None, amount=None, symbol = None, params = {"rate": 0.002})

#### Margin Loan Structure

In [ ]:
margin_loan_structure = {
    id: '1234323',                          # integer, the transaction id
    currency: 'USDT',                       # string, the currency that is borrowed or repaid
    amount: 5.81,                           # float, the amount of currency that was borrowed or repaid
    symbol: 'BTC/USDT:USDT',                # string, unified market symbol
    timestamp: 1648699200000,               # integer, the timestamp of when the transaction was made
    datetime: '2022-03-31T04:00:00.000Z',   # string, the datetime of when the transaction was made
    info: { ... }                           # Unparsed exchange response
}